用于处理蓝图Track生成的坐标点冗余数据

In [1]:
# 输入数据表格名

sheet_name = 'DS3-50-0'

定义八叉树数据结构

In [2]:
import sys
sys.setrecursionlimit(1000000)
MAX_OBJECTS_PER_CUBE = 10
DIRLOOKUP = {"11":0, "7":1, "5":2, "1":3, "-1":4, "-5":5, "-7":6, "-11":7}
#### End Globals ####

class OctNode:
    # New Octnode Class, can be appended to as well i think
    def __init__(self, position, size, data):
        # OctNode Cubes have a position and size
        # position is related to, but not the same as the objects the node contains.
        self.position = position
        self.size = size
        self.isLeafNode = True
        # store our object, typically this will be one, but maybe more
        self.data = data
        # might as well give it some emtpy branches while we are here.
        self.branches = [None, None, None, None, None, None, None, None]

        # 这部分先不用管，但是对BVH应该是有用的
        self.ldb = (position[0] - (size / 2), position[1] - (size / 2), position[2] - (size / 2))
        self.ruf = (position[0] + (size / 2), position[1] + (size / 2), position[2] + (size / 2))
        

class Octree:
    def __init__(self, worldSize):
        self.root = self.addNode((0,0,0), worldSize, [])
        self.worldSize = worldSize

    def addNode(self, position, size, objects):
        # 创建一个叶子节点
        return OctNode(position, size, objects)

    def insertNode(self, root, size, parent, objData):
        # 在这个函数里面objData为外部待加载数据，parent永远是当前节点，root是当前节点的子节点，size是正方形空间分割尺寸。
        if root == None:
            # 这个是针对self.branches = [None, None, None, None, None, None, None, None]
            # 当前节点走到branches中的任何一个元素，它都是None
            pos = parent.position
            offset = size / 2
            branch = self.findBranch(parent, objData.position)
            # new center = parent position + (branch direction * offset)
            newCenter = (0,0,0)
            # 看我的就对了
            if branch == 0:
                # 右上前
                newCenter = (pos[0] + offset, pos[1] + offset, pos[2] + offset )
                
            elif branch == 1:
                # 右下前
                newCenter = (pos[0] + offset, pos[1] + offset, pos[2] - offset )
                
            elif branch == 2:
                # 右上后
                newCenter = (pos[0] + offset, pos[1] - offset, pos[2] + offset )
                
            elif branch == 3:
                # 右下后
                newCenter = (pos[0] + offset, pos[1] - offset, pos[2] - offset )

            elif branch == 4:
                # 左上前
                newCenter = (pos[0] - offset, pos[1] + offset, pos[2] + offset )

            elif branch == 5:
                # 左下前
                newCenter = (pos[0] - offset, pos[1] + offset, pos[2] - offset )
                
            elif branch == 6:
                # 左上后
                newCenter = (pos[0] - offset, pos[1] - offset, pos[2] + offset )

            elif branch == 7:
                # 左下后
                newCenter = (pos[0] - offset, pos[1] - offset, pos[2] - offset )
            
            # 创建一个叶子节点，这里并不是独立的，因为它是parent当前节点的子节点 branches[branch]。
            return self.addNode(newCenter, size, [objData])
    
    
        elif root.position != objData.position and root.isLeafNode == False:
            # 在递归过程中，root就是当前节点，objData是待添加数据
            branch = self.findBranch(root, objData.position)
            # 所有子节点的newSize都是父节点的1/2尺寸
            newSize = root.size / 2
            
            root.branches[branch] = self.insertNode(root.branches[branch], newSize, root, objData)
        
        elif root.isLeafNode:
            
            # 如果当前节点是叶子节点（没有子节点）的时候，对该节点进行加载外部数据。
            # 第一个外部数据肯定是从这里开始进行运行。
            if len(root.data) < MAX_OBJECTS_PER_CUBE:
                # 当前节点的容量小于规定数量： MAX_OBJECTS_PER_CUBE时，直接把外部数据加载到data变量中。
                root.data.append(objData)
            
            elif len(root.data) == MAX_OBJECTS_PER_CUBE: 
                # 当一个节点，它的容量大于规定数量：MAX_OBJECTS_PER_CUBE时，就对所在空间进行再细分成八个部分，
                # 此时，它从叶子节点成了一个有子节点的节点，因此isLeafNode = False。因此要对data中的数据取出来，
                # 把data中每个数据（三维空间点）重新找在这个新节点中的branch是多少，找到后把该三维空间点对应的信息
                # 提取出来，创建一个叶子节点。以此为循环，进行递归。每一次划分，size/2。
                root.data.append(objData)
                objList = root.data
                root.data = None
                root.isLeafNode = False
                newSize = root.size / 2
                # print ("Subdividing Node sized at: " + str(root.size) + " at " + str(root.position))
                
                for ob in objList:
                    branch = self.findBranch(root, ob.position)
                    # 第一次运行到这里时，当前节点的branches数组目前还是 [None,None,None,None,None,None,None,None]
                    
                    root.branches[branch] = self.insertNode(root.branches[branch], newSize, root, ob)
        return root

    def findBranch(self, root, position):
        vec1 = root.position
        vec2 = position
        result = 0
        for i in range(3):
            if vec1[i] >= vec2[i]: # 待加数据位置小于当前节点位置：取负值
                result += (-12 / (i + 1) / 2)
            else:
                result += (12 / (i + 1) / 2)
        result = DIRLOOKUP[str(int(result))]
        return result
    
    def findPosition(self, root, position):
        # Basic collision lookup that finds the leaf node containing the specified position
        # Returns the child objects of the leaf, or None if the leaf is empty or none
        if root == None:
            return None
        elif root.isLeafNode:
            return root.data
        else:
            branch = self.findBranch(root, position)
            return self.findPosition(root.branches[branch], position)



In [3]:
import pandas as pd


data = pd.read_excel(sheet_name + '.xlsx')

In [4]:
import math


def dist(x, y):
    
    temp = 0
    for i in range(3):
        temp += (x[i] - y[i])**2

    return math.sqrt(temp)


# x = [-6870.934082,	25916.431641,	21502.082031]
# y = [-6636.102051,	25810.763672,	21522.029297]
# z = [-6558.505859,	25791.298828,	21522.205078]
# a = [-6480.908691,	25771.833984,	21522.378906]
# dist(x,y)

# 计算平均相邻距离

sum = 0
cnt = 0


datalist = []

for index,row in data.iterrows():
    datalist.append((row['X'], row['Y'], row['Z']))

for i in range(len(datalist)-1):
    distnum = dist(datalist[i],datalist[i+1])
    sum += distnum
    cnt += 1

print (sum/cnt)
    

# 计算截断位置

for dist_threshold in range(70,90):

    cnt = 0
    darkTree = Octree(100000.0000)

    for index, row in data.iterrows():
        # 定义新点
        name = "Node__" + str(index)
        pos = (row['X'], row['Y'], row['Z'])
        newPoint = TestObject(name, pos)

        # 检查临近点
        # 如果阈值范围内有点则跳过
        result = darkTree.findPosition(darkTree.root, pos)

        if result == None:
            darkTree.insertNode(darkTree.root, 100000.000, darkTree.root, newPoint)
            cnt += 1
            continue

        nearFlag = False    
        for resPoint in result:
            if dist(resPoint.position, pos) > dist_threshold:
                continue
            else:
                nearFlag = True

        if nearFlag: continue

        darkTree.insertNode(darkTree.root, 100000.000, darkTree.root, newPoint)
        cnt += 1


    End = time.time() - Start
    # print("距离为"+str(dist_threshold)+"时, 图中有点"+str(cnt)+"个")
    print(dist_threshold,"\t",cnt)



In [5]:
import time

class PointObject:
    def __init__(self, name, position):
        self.name = name
        self.position = position

# Number of objects we intend to add.
NUM_TEST_OBJECTS = 2000

# Number of collisions we're going to test
NUM_COLLISION_LOOKUPS = 2000

# Insert some random objects and time it
Start = time.time()

# Create a new octree, size of world
darkTree = Octree(100000.0000)
dist_threshold = 80
cnt = 0

for index, row in data.iterrows():
    # 定义新点
    name = "Node__" + str(index)
    pos = (row['X'], row['Y'], row['Z'])
    newPoint = PointObject(name, pos)

    # 检查临近点
    # 如果阈值范围内有点则跳过
    result = darkTree.findPosition(darkTree.root, pos)

    if result == None:
        darkTree.insertNode(darkTree.root, 100000.000, darkTree.root, newPoint)
        cnt += 1
        continue

    nearFlag = False    
    for resPoint in result:
        if dist(resPoint.position, pos) > dist_threshold:
            continue
        else:
            nearFlag = True

    if nearFlag: continue

    darkTree.insertNode(darkTree.root, 100000.000, darkTree.root, newPoint)
    cnt += 1


End = time.time() - Start


print(cnt)
# print some results.
print (str(NUM_TEST_OBJECTS) + "-Node Tree Generated in " + str(End) + " Seconds")
print ("Tree Leaves contain a maximum of " + str(MAX_OBJECTS_PER_CUBE) + " objects each.")




4705
2000-Node Tree Generated in 0.5120949745178223 Seconds
Tree Leaves contain a maximum of 10 objects each.


In [6]:


returnPointList = []
cnt = 0

for index, row in data.iterrows():
    pos = (row['X'], row['Y'], row['Z'])
    
    result = darkTree.findPosition(darkTree.root, pos)
  
    if result == None:
        continue

    isflag = False
    for resPoint in result:
        if dist(resPoint.position, pos) < 0.1:
            isflag = True
            continue

    if isflag: 
        returnPointList.append(pos)
        cnt += 1

    
print("Remain "+str(cnt)+" Points")


name=('X','Y','Z')
df = pd.DataFrame(columns=name, data=returnPointList)
print(df)
df.to_excel(sheet_name + '_return' + '.xlsx')





Remain 4706 Points
                 X             Y             Z
0     -6870.934082  25916.431641  21502.082031
1     -6636.102051  25810.763672  21522.029297
2     -6558.505859  25791.298828  21522.205078
3     -6480.908691  25771.833984  21522.378906
4     -6403.311523  25752.369141  21522.554688
...            ...           ...           ...
4701 -18216.722656   5847.771484 -32485.179688
4702 -18278.515625   5912.658203 -32485.511719
4703 -18340.785156   5978.046875 -32485.843750
4704 -18405.451172   6045.949219 -32486.191406
4705 -18461.494141   6104.800781 -32486.492188

[4706 rows x 3 columns]
